<a href="https://colab.research.google.com/github/AntonDozhdikov/RL_OTUS_Learn/blob/main/Car_racing_v3_A2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title установка драйверов
# Команда apt-get update обновляет индекс пакетов, чтобы получить последнюю информацию о доступных версиях пакетов.
!apt-get update > /dev/null 2>&1

# Установка необходимых пакетов: xvfb (виртуальный фреймбуфер X), Python OpenGL и ffmpeg (для обработки видео).
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Установка пакетов для разработки на C/C++ и Python: swig (интерфейс между языками программирования), build-essential (инструменты сборки), python-dev и python3-dev (заголовочные файлы и библиотеки для Python).
!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1

# Установка утилит для работы с X-сервером, таких как xrandr и xdpyinfo.
!apt-get install x11-utils > /dev/null 2>&1

# Установка виртуального фреймбуфера X (xvfb), который позволяет запускать графические приложения без физического дисплея.
!apt-get install xvfb > /dev/null 2>&1

In [2]:
# @title установка библиотек
#%%capture
# Устанавливает библиотеку rarfile для работы с архивами RAR.
!pip install rarfile --quiet

# Устанавливает пакет stable-baselines3 с дополнительными зависимостями.
!pip install stable-baselines3[extra] --quiet

# Устанавливает библиотеку Ale-Py для работы с Atari играми.
!pip install ale-py --quiet

# Устанавливает пакет gym с поддержкой Box2D для физических симуляций.
!pip install gym[box2d] --quiet

# Устанавливает PyVirtualDisplay для создания виртуальных дисплеев. Страшно кривая вещь!!! Ни в колабе ни на машине не работает
!pip install pyvirtualdisplay --quiet

# Устанавливает Pyglet для работы с графикой и мультимедиа.
!pip install pyglet --quiet

# Устанавливает PyGame для разработки игр.
!pip install pygame --quick

# Устанавливает Minigrid для создания простых gridworld сред.
!pip install minigrid --quiet

# Устанавливает SWIG для взаимодействия между различными языками программирования.
!pip install -q swig --quiet

# Устанавливает Gymnasium с поддержкой Box2D для физических симуляций.
!pip install -q gymnasium[box2d] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This

In [3]:
# Импорт необходимых библиотек
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.normal import Normal
from collections import deque
import matplotlib.pyplot as plt
from datetime import datetime
import os
import warnings

In [4]:
# Фильтрация предупреждений
warnings.filterwarnings('ignore', category=UserWarning)

# Гиперпараметры
ENV_NAME = "CarRacing-v3"       # Установка верссии среды
LEARNING_RATE_ACTOR = 0.0001    # Скорость обучения для нейронной сети актёра (Actor Network)
LEARNING_RATE_CRITIC = 0.0002   # Скорость обучения для нейронной сети критика (Critic Network)
GAMMA = 0.99                    # Коэффициент дисконтирования будущих вознаграждений
EPISODES = 300                  # Общее количество эпизодов для тренировки агента
BATCH_SIZE = 64                 # Размер мини-пакета данных для обновления весов модели
SAVE_INTERVAL = 30              # Интервал шагов для сохранения модели
HIDDEN_SIZE = 256               # Размер скрытого слоя в нейронных сетях
LOG_INTERVAL = 10               # Интервал шагов для логгирования результатов
IMAGE_SIZE = (96, 96)           # Размер входного изображения для обработки моделью

In [5]:
# Папки для сохранения
MODEL_DIR = "saved_models"      # Директория для сохранения моделей
LOG_DIR = "logs"                # Директория для сохранения логов
os.makedirs(MODEL_DIR, exist_ok=True)     # Создаем директорию MODEL_DIR, если она еще не существует
os.makedirs(LOG_DIR, exist_ok=True)       # Создаем директорию LOG_DIR, если она еще не существует

In [6]:
# Предобработка изображения
def preprocess_image(image):
    # Конвертация в оттенки серого и изменение размера
    image = image.mean(axis=2)  # Усреднение по RGB каналам
    image = image[::2, ::2]     # Уменьшение разрешения до 48x48
    image = image / 255.0       # Нормализация [0, 1]
    return image[np.newaxis, ...]  # Добавление размерности канала

In [7]:
# Архитектура сети актора
class Actor(nn.Module):
    def __init__(self, action_dim):
        super(Actor, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 8, stride=4),          # Первый сверточный слой с 32 фильтрами размером 8x8 и шагом 4
            nn.ReLU(),                              # Функция активации ReLU
            nn.Conv2d(32, 64, 4, stride=2),         # Второй сверточный слой с 64 фильтрами размером 4x4 и шагом 2
            nn.ReLU(),                              # Функция активации ReLU
            nn.Conv2d(64, 64, 3, stride=1),         # Третий сверточный слой с 64 фильтрами размером 3x3 и шагом 1
            nn.ReLU(),                              # Функция активации ReLU
            nn.Flatten()                            # Слой для преобразования многомерного тензора в одномерный
        )
        conv_out_size = self._get_conv_out_size((1, 48, 48))  # Определение размера выходного сигнала сверток
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, HIDDEN_SIZE),  # Полносвязный слой с числом нейронов равным значению HIDDEN_SIZE
            nn.ReLU(),                              # Функция активации ReLU
            nn.Linear(HIDDEN_SIZE, action_dim),     # Выходной полносвязный слой с количеством нейронов равным числу возможных действий
            nn.Tanh()                               # Функция активации Tanh для ограничения выходных значений в диапазоне [-1, 1]
        )
        self.log_std = nn.Parameter(torch.zeros(action_dim))  # Логарифм стандартного отклонения для нормального распределения

    def _get_conv_out_size(self, shape):
        dummy = torch.zeros(1, *shape)                   # Создание нулевого тензора для определения размера выхода сверток
        return self.conv(dummy).view(1, -1).size(1)      # Возвращает размерность выходного сигнала сверток

    def forward(self, x):
        x = self.conv(x)                                 # Пропускаем данные через сверточную часть сети
        mu = self.fc(x)                                  # Получаем среднее значение для нормального распределения
        std = torch.exp(self.log_std)                    # Вычисляем стандартное отклонение
        return mu, std                                   # Возвращаем среднее и стандартное отклонение


In [8]:
# Архитектура сети критика
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()                  # Вызываем конструктор родительского класса
        self.conv = nn.Sequential(                       # Последовательность слоев свертки
            nn.Conv2d(1, 32, 8, stride=4),               # Первый сверточный слой с 32 фильтрами размером 8x8 и шагом 4
            nn.ReLU(),                                   # Функция активации ReLU
            nn.Conv2d(32, 64, 4, stride=2),              # Второй сверточный слой с 64 фильтрами размером 4x4 и шагом 2
            nn.ReLU(),                                   # Функция активации ReLU
            nn.Conv2d(64, 64, 3, stride=1),              # Третий сверточный слой с 64 фильтрами размером 3x3 и шагом 1
            nn.ReLU(),                                   # Функция активации ReLU
            nn.Flatten()                                 # Слой для преобразования многомерного тензора в одномерный
        )
        conv_out_size = self._get_conv_out_size((1, 48, 48))  # Определение размера выходного сигнала сверток
        self.fc = nn.Sequential(                          # Последовательность полносвязных слоев
            nn.Linear(conv_out_size, HIDDEN_SIZE),        # Полносвязный слой с числом нейронов равным значению HIDDEN_SIZE
            nn.ReLU(),                                    # Функция активации ReLU
            nn.Linear(HIDDEN_SIZE, 1)                     # Выходной полносвязный слой с одним нейроном для оценки значения состояния
        )

    def _get_conv_out_size(self, shape):
        dummy = torch.zeros(1, *shape)                    # Создание нулевого тензора для определения размера выхода сверток
        return self.conv(dummy).view(1, -1).size(1)       # Возвращает размерность выходного сигнала сверток

    def forward(self, x):
        x = self.conv(x)                                  # Пропускаем данные через сверточную часть сети
        return self.fc(x)                                 # Пропускаем результат через полносвязные слои и возвращаем оценку значения состояния


In [9]:
# Агент A2C
class A2CAgent:
    def __init__(self, env):
        self.env = env                                # Сохраняем среду окружения
        self.action_dim = env.action_space.shape[0]   # Определяем размерность пространства действий

        # Инициализация сетей
        self.actor = Actor(self.action_dim)           # Создаем экземпляр сети актора
        self.critic = Critic()                        # Создаем экземпляр сети критика

        # Оптимизаторы
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=LEARNING_RATE_ACTOR)  # Оптимизатор для актора
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=LEARNING_RATE_CRITIC)  # Оптимизатор для критика

        # Буферы
        self.states = []                      # Список состояний
        self.actions = []                     # Список действий
        self.rewards = []                     # Список наград
        self.dones = []                       # Список индикаторов завершения эпизода
        self.values = []                      # Список оценочных значений состояний

        # Логирование
        self.log = {'episode': [], 'reward': [], 'length': []}  # Словарь для хранения информации об эпизодах
        self.best_reward = -np.inf            # Лучшая награда, изначально равна минус бесконечности

    def get_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Преобразуем состояние в тензор PyTorch
        with torch.no_grad():  # Отключаем вычисление градиентов для выбора действия
            mu, std = self.actor(state_tensor)  # Получаем среднее и стандартное отклонение из сети актёра
            dist = Normal(mu, std)              # Создаем нормальное распределение
            action = dist.sample()              # Генерируем случайное действие из распределения
        return action.numpy()[0], dist, self.critic(state_tensor).item()  # Возвращаем выбранное действие, распределение и оценку состояния
    def update(self):
        states = torch.FloatTensor(np.array(self.states))  # Преобразуем список состояний в тензор PyTorch
        actions = torch.FloatTensor(np.array(self.actions))  # Преобразуем список действий в тензор PyTorch
        rewards = torch.FloatTensor(np.array(self.rewards))  # Преобразуем список наград в тензор PyTorch
        dones = torch.FloatTensor(np.array(self.dones))  # Преобразуем список индикаторов завершения в тензор PyTorch

        # Пересчитываем значения через критика с включенными градиентами
        current_values = self.critic(states).squeeze()  # Оценка текущих состояний сетью критика

        # Расчёт возвратов с дисконтированием
        returns = []  # Список для накопления возвратов
        R = 0  # Начальное значение возврата
        for r, done in zip(reversed(rewards.numpy()), reversed(dones.numpy())):  # Проходим назад по спискам наград и завершений
            R = r + GAMMA * R * (1 - done)  # Рассчитываем возврат с учётом коэффициента дисконтирования
            returns.insert(0, R)  # Добавляем рассчитанный возврат в начало списка
        returns = torch.FloatTensor(returns)  # Преобразуем список возвратов в тензор PyTorch

        # Нормализация преимуществ
        advantages = returns - current_values  # Разница между возвратами и текущими значениями (преимущества)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)  # Нормализуем преимущества

        # Обновление критика
        self.optimizer_critic.zero_grad()  # Сбрасываем накопленные градиенты оптимизатора критика
        value_loss = F.mse_loss(current_values, returns)  # Рассчитываем ошибку как среднеквадратичное отклонение между текущими значениями и возвратами
        value_loss.backward()  # Выполняем обратное распространение ошибки
        self.optimizer_critic.step()  # Применяем обновлённые веса к сети критика

        # Обновление актёра
        self.optimizer_actor.zero_grad()  # Сбрасываем накопленные градиенты оптимизатора актора
        mu, std = self.actor(states)  # Получаем среднее и стандартное отклонение из сети актора
        dist = Normal(mu, std)  # Создаём нормальное распределение
        log_probs = dist.log_prob(actions).sum(-1)  # Рассчитываем логарифмические вероятности действий
        actor_loss = -(log_probs * advantages.detach()).mean()  # Рассчитываем потерю актора, умножая логарифмы вероятностей на нормализованные преимущества
        actor_loss.backward()  # Выполняем обратное распространение ошибки
        self.optimizer_actor.step()  # Применяем обновлённые веса к сети актора

        self._clear_buffers()  # Очищаем буферы для следующего шага
    def _clear_buffers(self):
        self.states.clear()       # Очистить список состояний
        self.actions.clear()      # Очистить список действий
        self.rewards.clear()      # Очистить список наград
        self.dones.clear()        # Очистить список индикаторов завершения
        self.values.clear()       # Очистить список оценочных значений

    def save_model(self, episode):
        torch.save({                                      # Сохраняем модель и сопутствующие данные
            'actor': self.actor.state_dict(),             # Сохраняем состояние сети актора
            'critic': self.critic.state_dict(),           # Сохраняем состояние сети критика
            'optimizer_actor': self.optimizer_actor.state_dict(),  # Сохраняем состояние оптимизатора актора
            'optimizer_critic': self.optimizer_critic.state_dict(),  # Сохраняем состояние оптимизатора критика
            'episode': episode                           # Сохраняем номер текущего эпизода
        }, f'{MODEL_DIR}/a2c_{ENV_NAME}_ep_{episode}.pth')  # Имя файла для сохранения модели

    def load_model(self, model_path):
        checkpoint = torch.load(model_path)  # Загружаем сохранённую модель
        self.actor.load_state_dict(checkpoint['actor'])  # Восстанавливаем состояние сети актора
        self.critic.load_state_dict(checkpoint['critic'])  # Восстанавливаем состояние сети критика
        self.optimizer_actor.load_state_dict(checkpoint['optimizer_actor'])  # Восстанавливаем состояние оптимизатора актора
        self.optimizer_critic.load_state_dict(checkpoint['optimizer_critic'])  # Восстанавливаем состояние оптимизатора критика
        return checkpoint['episode']  # Возвращаем номер последнего эпизода

    def plot_training(self):
        plt.figure(figsize=(10,5))  # Создаём фигуру для графика
        plt.plot(self.log['episode'], self.log['reward'], label='Средняя награда')  # Строим график средней награды по эпизодам
        plt.plot(self.log['episode'],  # Строим график скользящего среднего
                 np.convolve(self.log['reward'], np.ones(100)/100, mode='same'),  # Скользящее среднее с окном 100
                 label='Скользящее среднее (100 эп)')
        plt.xlabel('Эпизоды')  # Подписываем ось X
        plt.ylabel('Суммарная награда')  # Подписываем ось Y
        plt.title('Динамика обучения')  # Указываем заголовок графика
        plt.legend()  # Показать легенду
        plt.grid()  # Включаем сетку
        plt.savefig(f'{LOG_DIR}/training_curve.png')  # Сохраняем график в файл
        plt.close()  # Закрываем текущий график


In [10]:
# Обучение агента
def train():
    env = gym.make(ENV_NAME, continuous=True)  # Создаем среду для обучения
    agent = A2CAgent(env)  # Инициализируем агента

    print("Старт обучения...")  # Сообщаем о начале обучения
    start_time = datetime.now()  # Запоминаем время начала обучения

    for ep in range(1, EPISODES+1):  # Основной цикл обучения по эпизодам
        state, _ = env.reset()  # Сбрасываем среду и получаем начальное состояние
        state = preprocess_image(state)  # Предварительная обработка изображения
        episode_reward = 0  # Переменная для подсчета общей награды за эпизод
        done = False  # Флаг завершения эпизода
        step = 0  # Счетчик шагов внутри эпизода

        while not done:  # Цикл выполнения одного эпизода
            # Выбор действия
            action, dist, value = agent.get_action(state)  # Получаем действие, распределение и оценку состояния

            # Шаг в среде
            next_state, reward, done, truncated, info = env.step(action)  # Делаем шаг в среде
            next_state = preprocess_image(next_state)  # Предварительная обработка следующего состояния

            # Обновление буферов
            agent.states.append(state)  # Добавляем текущее состояние в буфер
            agent.actions.append(action)  # Добавляем выполненное действие в буфер
            agent.rewards.append(reward)  # Добавляем полученную награду в буфер
            agent.dones.append(done)  # Добавляем индикатор завершения в буфер
            agent.values.append(value)  # Добавляем оценку состояния в буфер

            state = next_state  # Обновляем текущее состояние
            episode_reward += reward  # Накапливаем общую награду за эпизод
            step += 1  # Увеличиваем счетчик шагов

            # Обновление сетей
            if step % BATCH_SIZE == 0 or done:  # Если достигли конца мини-пакета или эпизода
                agent.update()  # Обновляем параметры сети

            if done or truncated:  # Если эпизод завершился или был прерван
                break  # Завершаем выполнение текущего эпизода

        # Логирование
        agent.log['episode'].append(ep)  # Добавляем номер эпизода в журнал
        agent.log['reward'].append(episode_reward)  # Добавляем суммарную награду за эпизод в журнал
        agent.log['length'].append(step)  # Добавляем длину эпизода в журнал

        # Сохранение лучшей модели
        if episode_reward > agent.best_reward:  # Если текущая награда лучше предыдущей
            agent.best_reward = episode_reward  # Обновляем лучшую награду
            agent.save_model(ep)  # Сохраняем модель
            print(f"Эпизод {ep}: Новая лучшая награда {episode_reward:.2f}")  # Сообщаем о новой лучшей награде

        # Вывод статистики
        if ep % LOG_INTERVAL == 0:  # Каждые LOG_INTERVAL эпизодов выводим статистику
            avg_reward = np.mean(agent.log['reward'][-LOG_INTERVAL:])  # Рассчитываем среднюю награду за последние LOG_INTERVAL эпизодов
            print(f"Эпизод {ep}, Средняя награда: {avg_reward:.2f}, Время: {datetime.now() - start_time}")  # Выводим информацию

    # Сохранение финальной модели и логирование
    agent.save_model(EPISODES)  # Сохраняем последнюю модель
    agent.plot_training()  # Строим график обучения
    env.close()  # Закрываем среду

if __name__ == "__main__":
    train()  # Запускаем обучение


Старт обучения...
Эпизод 1: Новая лучшая награда -51.72
Эпизод 6: Новая лучшая награда -45.95
Эпизод 7: Новая лучшая награда -43.61
Эпизод 10: Новая лучшая награда -35.11
Эпизод 10, Средняя награда: -49.31, Время: 0:02:45.870070
Эпизод 12: Новая лучшая награда -32.66
Эпизод 15: Новая лучшая награда -7.75
Эпизод 18: Новая лучшая награда 4.65
Эпизод 20, Средняя награда: -43.55, Время: 0:05:32.249379
Эпизод 30, Средняя награда: -51.68, Время: 0:08:22.894400
Эпизод 40, Средняя награда: -76.02, Время: 0:11:00.600902
Эпизод 50, Средняя награда: -77.61, Время: 0:13:54.036092
Эпизод 60, Средняя награда: -75.93, Время: 0:16:49.259940
Эпизод 70, Средняя награда: -73.40, Время: 0:19:37.170477
Эпизод 80, Средняя награда: -69.90, Время: 0:22:23.550247
Эпизод 90, Средняя награда: -61.67, Время: 0:25:14.911917
Эпизод 100, Средняя награда: -73.87, Время: 0:28:03.474330
Эпизод 110, Средняя награда: -70.54, Время: 0:30:58.795128
Эпизод 120, Средняя награда: -73.49, Время: 0:33:52.973238
Эпизод 130, Сред